In [ ]:
import sys
from pathlib import Path

# Jupyter Notebook 环境下的路径处理
try:
    root_path = next(
        (
            p
            for p in Path(__file__).resolve().parents
            if (p / "pyproject.toml").is_file()
        ),
        None,
    )
except NameError:
    current_dir = Path.cwd()
    root_path = next(
        (
            p
            for p in [current_dir] + list(current_dir.parents)
            if (p / "pyproject.toml").is_file()
        ),
        None,
    )

if root_path:
    sys.path.insert(0, str(root_path))

In [ ]:
from py_entry.Test.backtest.strategies import get_strategy
from py_entry.data_conversion.backtest_runner import BacktestRunner

strategy = get_strategy("reversal_extreme")
br = (
    BacktestRunner()
    .setup(
        data_source=strategy.data_config,
        indicators_params=strategy.indicators_params,
        signal_params=strategy.signal_params,
        backtest_params=strategy.backtest_params,
        signal_template=strategy.signal_template,
        engine_settings=strategy.engine_settings,
    )
    .run()
    .format_results_for_export(export_index=0, dataframe_format="csv")
)
assert br.results is not None, "检测到None"
result = br.results[0]

In [ ]:
# import polars as pl
# pl.Config.set_tbl_rows(500)
result.backtest_result

In [ ]:
br.diagnose_states(result_index=0, print_summary=True)

In [ ]:
from py_entry.data_conversion.file_utils import DisplayConfig, DashboardOverride

config = DisplayConfig(
    embed_data=False,
    width="100%",
    aspect_ratio="16/9",
    override=DashboardOverride(
        show=["0,0,0,1"],
        showInLegend=["0,0,0,1"],
        showRiskLegend="1,1,1,1",
        showLegendInAll=True,
    ).to_dict(),
)

br.display_dashboard(config=config)

In [ ]:
import json
from pathlib import Path

json_path = Path(Path.cwd() / "performance.json")
with open(json_path, "w") as f:
    json.dump(result.performance, f, indent=2, sort_keys=True)
print(f"✅ 已保存 performance.json: {json_path}")